In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Overview

In this notebook, you will learn how to load, explore, visualize, and pre-process a time-series dataset. The output of this notebook is a processed dataset that will be used in following notebooks to build a machine learning model.

### Dataset

[CTA - Ridership - Daily Boarding Totals](https://data.cityofchicago.org/Transportation/CTA-Ridership-Daily-Boarding-Totals/6iiy-9s97): This dataset shows systemwide boardings for both bus and rail services provided by Chicago Transit Authority, dating back to 2001.

### Objective

The goal is to forecast future transit ridership in the City of Chicago, based on previous ridership.

## Install packages and dependencies

Restarting the kernel may be required to use new packages.

In [1]:
%pip install -U statsmodels scikit-learn --user

Note: you may need to restart the kernel to use updated packages.


**Note:** To restart the Kernel, navigate to Kernel > Restart Kernel... on the Jupyter menu.

### Import libraries and define constants

In [1]:
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [6]:
# Enter your project and region. Then run the  cell to make sure the
# Cloud SDK uses the right project for all the commands in this notebook.

PROJECT = 'deep-learning-platform-305222' # REPLACE WITH YOUR PROJECT NAME 
REGION = 'us-central-1' # REPLACE WITH YOUR REGION e.g. us-central1

#Don't change the following command - this is to check if you have changed the project name above.
assert PROJECT != 'your-project-name', 'Don''t forget to change the project variables!'

In [12]:
target = 'y' # The variable you are predicting
target_description = 'Total Rides' # A description of the target variable
features = {'day_type': 'Day Type'} # Weekday = W, Saturday = A, Sunday/Holiday = U
ts_col = 'ds' # The name of the column with the date field

raw_data_file = 'https://data.cityofchicago.org/api/views/6iiy-9s97/rows.csv?accessType=DOWNLOAD'
processed_file = 'nyc_311.csv' # Which file to save the results to

## Load data

## Explore data

In [7]:
# Print the top 5 rows

from google.cloud import bigquery as bq

sql = """
SELECT * FROM `bigquery-public-data.new_york_311.311_service_requests` LIMIT 5
"""

client = bq.Client(project=PROJECT)
df = client.query(sql).to_dataframe()

df.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pickup_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,24971566,2013-02-10 16:32:40+00:00,2013-02-11 12:20:54+00:00,DFTA,Department for the Aging,Housing - Low Income Senior,N/A,Senior Address,10454,None,...,None,None,None,None,None,None,None,NaN,NaN,None
1,25003455,2013-02-15 12:31:52+00:00,2013-02-20 12:38:24+00:00,DFTA,Department for the Aging,Housing - Low Income Senior,N/A,Senior Address,10454,None,...,None,None,None,None,None,None,None,NaN,NaN,None
2,25058152,2013-02-23 09:47:54+00:00,2013-02-23 10:51:21+00:00,NYPD,New York City Police Department,Homeless Encampment,N/A,Residential Building/House,10455,720 WESTCHESTER AVENUE,...,None,None,None,None,None,None,None,40.817187,-73.906616,"(40.817186555273494, -73.90661626221494)"
3,25065929,2013-02-25 10:14:35+00:00,2013-02-27 11:59:35+00:00,DFTA,Department for the Aging,Housing - Low Income Senior,N/A,Senior Address,10454,None,...,None,None,None,None,None,None,None,NaN,NaN,None
4,25065931,2013-02-25 15:43:04+00:00,2013-02-27 12:13:45+00:00,DFTA,Department for the Aging,Housing - Low Income Senior,N/A,Senior Address,10451,None,...,None,None,None,None,None,None,None,NaN,NaN,None


### TODO 1: Analyze the patterns

* Is ridership changing much over time?
* Is there a difference in ridership between the weekday and weekends?
* Is the mix of bus vs rail ridership changing over time?

In [14]:
# Print the top 5 rows

from google.cloud import bigquery as bq

sql = """
SELECT
  COUNT(unique_key) as y,
  DATE_TRUNC(DATE(created_date), month) as ds  
FROM `bigquery-public-data.new_york_311.311_service_requests`
GROUP by ds ORDER BY ds asc
"""

client = bq.Client(project=PROJECT)
df = client.query(sql).to_dataframe()

df.head()

,y,ds
0,182117,2010-01-01
1,159489,2010-02-01
2,198639,2010-03-01
3,162854,2010-04-01
4,158039,2010-05-01


## Export data

This will generate a CSV file, which you will use in the next labs of this quest.
Inspect the CSV file to see what the data looks like.

In [16]:
df[[target]].to_csv("nyc_311_monthly.csv", index=True, index_label=ts_col)

## Conclusion

You've successfully completed the exploration and visualization lab.
You've learned how to:
* Create a query that groups data into a time series
* Visualize data
* Decompose time series into trend and seasonal components